In [ ]:
import os
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import re

google_api_key = os.environ["GOOGLE_API_KEY"]

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
)

# State definition
class OrderState(TypedDict):
    user_query: str
    route: str
    products_to_search: List[str]
    search_results: Dict[str, Any]
    wants_to_order: bool
    quantity: int
    messages: List[str]

# Mock product database
PRODUCT_DB = {
    "apple": {"name": "Apple", "price": 1.50, "in_stock": True},
    "orange": {"name": "Orange", "price": 2.00, "in_stock": True},
    "pineapple": {"name": "Pineapple", "price": 5.00, "in_stock": False},
    "banana": {"name": "Banana", "price": 1.00, "in_stock": True},
    "grape": {"name": "Grape", "price": 3.00, "in_stock": True},
    "milk": {"name": "Milk", "price": 2.50, "in_stock": True},
    "bread": {"name": "Bread", "price": 3.00, "in_stock": True}
}

def mock_product_search(product_name: str) -> Dict[str, Any]:
    """Mock function to search for products"""
    print(f"🔍 Searching for: {product_name}")
    product_key = product_name.lower().strip()
    if product_key in PRODUCT_DB:
        return PRODUCT_DB[product_key]
    return {"name": product_name, "price": 0, "in_stock": False, "found": False}

def route_query(state: OrderState) -> OrderState:
    """Route user query to appropriate handler"""
    print("🚦 Routing query...")
    query = state["user_query"]
    
    prompt = f"""Analyze this user query and determine if it's related to ordering products or just greeting/irrelevant topics.

Query: "{query}"

Respond with either "order" or "greeting" only."""
    
    try:
        response = llm.invoke(prompt)
        route = response.content.strip().lower()
        state["route"] = "order" if "order" in route else "greeting"
        state["messages"] = [f"Routing query to: {state['route']}"]
        print(f"✅ Routed to: {state['route']}")
    except Exception as e:
        print(f"❌ Error in routing: {e}")
        state["route"] = "greeting"
        state["messages"] = ["Error in routing, defaulting to greeting"]
    
    return state

def handle_greeting(state: OrderState) -> OrderState:
    """Handle greeting and irrelevant topics"""
    print("👋 Handling greeting...")
    query = state["user_query"]
    
    prompt = f"""The user said: "{query}"

Provide a friendly response. If it's a greeting, greet them back and mention you can help with ordering products.
Keep it brief and helpful."""
    
    try:
        response = llm.invoke(prompt)
        state["messages"].append(f"Response: {response.content}")
        print("✅ Greeting handled")
    except Exception as e:
        print(f"❌ Error in greeting: {e}")
        state["messages"].append("Hello! I can help you order products. What would you like?")
    
    return state

def extract_products(state: OrderState) -> OrderState:
    """Extract and break down products from user query using intelligent LLM parsing"""
    print("📦 Extracting products...")
    query = state["user_query"]
    
    prompt = f"""User Query: "{query}"

Instructions:
- Identify all distinct product mentions from the user message
- Generate exactly one simple search term per item, staying close to user phrasing
- Focus on food items, groceries, or products that can be ordered
- Normalize to singular form (e.g., "apples" -> "apple")
- Return output as a JSON array of strings

Examples:
- "Do you have apples, oranges and pineapples?" -> ["apple", "orange", "pineapple"]
- "I want to buy bananas" -> ["banana"]
- "Can I order some milk and bread?" -> ["milk", "bread"]
- "Hello how are you?" -> []

Return only the JSON array, nothing else:"""
    
    try:
        response = llm.invoke(prompt)
        response_text = response.content.strip()
        print(f"🤖 LLM response: {response_text}")
        
        # Try to extract JSON from response
        if response_text.startswith('[') and response_text.endswith(']'):
            products = json.loads(response_text)
        else:
            # Try to find JSON in the response
            json_match = re.search(r'\[.*?\]', response_text)
            if json_match:
                products = json.loads(json_match.group())
            else:
                products = []
        
        state["products_to_search"] = products if isinstance(products, list) else []
        state["messages"].append(f"Products to search: {state['products_to_search']}")
        print(f"✅ Extracted products: {state['products_to_search']}")
    except Exception as e:
        print(f"❌ Error extracting products: {e}")
        state["products_to_search"] = []
        state["messages"].append("Could not extract products from query")
    
    return state

def search_products(state: OrderState) -> OrderState:
    """Search for each product separately using the extracted products list"""
    print("🔎 Searching products...")
    products = state["products_to_search"]
    search_results = {}
    
    if not products:
        state["messages"].append("No products found to search.")
        state["search_results"] = {}
        return state
    
    print(f"🔍 Will search for {len(products)} products: {products}")
    
    # Search each product individually
    for product in products:
        print(f"   → Searching for: {product}")
        result = mock_product_search(product)
        search_results[product] = result
    
    state["search_results"] = search_results
    
    # Create summary message
    summary = "Search Results:\n"
    for product, result in search_results.items():
        if result.get("in_stock", False):
            summary += f"✅ {result['name']}: ${result['price']} (Available)\n"
        else:
            summary += f"❌ {product.title()}: Not available\n"
    
    state["messages"].append(summary)
    print("✅ Products searched")
    
    return state

def ask_order_confirmation(state: OrderState) -> OrderState:
    """Ask if user wants to order available products"""
    print("❓ Asking order confirmation...")
    available_products = [
        f"{result['name']} (${result['price']})" 
        for result in state["search_results"].values() 
        if result.get("in_stock", False)
    ]
    
    if not available_products:
        state["messages"].append("Sorry, none of the requested products are available.")
        state["wants_to_order"] = False
        print("❌ No products available")
        return state
    
    prompt = f"""Available products: {', '.join(available_products)}

Ask the user if they want to order any of these products. Keep it simple and friendly."""
    
    try:
        response = llm.invoke(prompt)
        state["messages"].append(response.content)
        
        # Actually ask user for input instead of simulating
        print(f"\n🤖 {response.content}")
        user_response = input("👤 Your response: ").strip().lower()
        state["messages"].append(f"User response: {user_response}")
        
        # Check if user wants to order
        wants_order = any(word in user_response for word in ["yes", "y", "sure", "ok", "order", "buy"])
        state["wants_to_order"] = wants_order
        
        if wants_order:
            print("✅ User wants to order!")
        else:
            print("❌ User doesn't want to order")
            
    except Exception as e:
        print(f"❌ Error in order confirmation: {e}")
        state["wants_to_order"] = False
    
    return state

def ask_quantity(state: OrderState) -> OrderState:
    """Ask for quantity"""
    print("🔢 Asking quantity...")
    available_products = [
        result['name'] 
        for result in state["search_results"].values() 
        if result.get("in_stock", False)
    ]
    
    if available_products:
        product_name = available_products[0]  # For simplicity, take first available
        quantity_question = f"How many {product_name}s would you like to order?"
        state["messages"].append(quantity_question)
        
        # Actually ask user for quantity instead of simulating
        print(f"\n🤖 {quantity_question}")
        try:
            user_input = input("👤 Enter quantity: ").strip()
            quantity = int(user_input)
            state["quantity"] = quantity
            state["messages"].append(f"User response: {quantity}")
            
            # Calculate total
            product_price = next(
                result["price"] for result in state["search_results"].values() 
                if result.get("in_stock", False)
            )
            total = product_price * state["quantity"]
            confirmation = f"Order confirmed: {state['quantity']} {product_name}s for ${total:.2f}"
            state["messages"].append(confirmation)
            print(f"✅ {confirmation}")
            
        except ValueError:
            print("❌ Invalid quantity entered, defaulting to 1")
            state["quantity"] = 1
            state["messages"].append("User response: 1 (default)")
            
        except Exception as e:
            print(f"❌ Error getting quantity: {e}")
            state["quantity"] = 1
            
        print("✅ Quantity processed")
    
    return state

def end_conversation(state: OrderState) -> OrderState:
    """End the conversation"""
    print("🏁 Ending conversation...")
    if state["route"] == "order" and not state.get("wants_to_order", False):
        state["messages"].append("No problem! Feel free to ask if you need anything else.")
    elif state["route"] == "order" and state.get("wants_to_order", False):
        state["messages"].append("Thank you for your order! Have a great day!")
    else:
        state["messages"].append("Feel free to ask if you'd like to order anything!")
    
    print("✅ Conversation ended")
    return state

# Define routing logic with simpler names
def route_decision(state: OrderState) -> str:
    result = "extract" if state["route"] == "order" else "greeting"
    print(f"🔀 Routing decision: {result}")
    return result

def order_decision(state: OrderState) -> str:
    has_available = any(
        result.get("in_stock", False) 
        for result in state["search_results"].values()
    )
    result = "confirm" if has_available else "end"
    print(f"🔀 Order decision: {result}")
    return result

def quantity_decision(state: OrderState) -> str:
    result = "qty" if state.get("wants_to_order", False) else "end"
    print(f"🔀 Quantity decision: {result}")
    return result

# Create the graph with shorter node names
def create_order_graph():
    print("🏗️ Building graph...")
    workflow = StateGraph(OrderState)
    
    # Add nodes with names that don't conflict with state keys
    workflow.add_node("router", route_query)
    workflow.add_node("greeting", handle_greeting)
    workflow.add_node("extract", extract_products)
    workflow.add_node("search", search_products)
    workflow.add_node("confirm", ask_order_confirmation)
    workflow.add_node("qty", ask_quantity)
    workflow.add_node("end", end_conversation)
    
    # Set entry point
    workflow.set_entry_point("router")
    
    # Add conditional edges with simpler function names
    workflow.add_conditional_edges("router", route_decision)
    workflow.add_edge("greeting", "end")
    workflow.add_edge("extract", "search")
    workflow.add_conditional_edges("search", order_decision)
    workflow.add_conditional_edges("confirm", quantity_decision)
    workflow.add_edge("qty", "end")
    workflow.add_edge("end", END)
    
    print("✅ Graph built successfully")
    return workflow.compile()

# Visualization function
def visualize_graph():
    """Visualize the graph with fallback options"""
    from IPython.display import Image, display
    
    try:
        graph = create_order_graph()
        
        # Try PNG first
        try:
            display(Image(graph.get_graph().draw_mermaid_png()))
            print("✅ Graph visualization displayed!")
            return
        except Exception as e:
            print(f"PNG failed: {e}")
        
        # Fallback to mermaid code
        mermaid_code = graph.get_graph().draw_mermaid()
        print("📋 Mermaid Code (paste at https://mermaid.live/):")
        print("="*60)
        print(mermaid_code)
        print("="*60)
        
    except Exception as e:
        print(f"❌ Visualization error: {e}")

# Example usage
def run_order_system(user_query: str):
    """Run the order system with a user query"""
    print(f"🚀 Starting order system...")
    print(f"User Query: {user_query}")
    print("=" * 50)
    
    try:
        graph = create_order_graph()
        
        initial_state = OrderState(
            user_query=user_query,
            route="",
            products_to_search=[],
            search_results={},
            wants_to_order=False,
            quantity=0,
            messages=[]
        )
        
        print("⚡ Executing graph...")
        result = graph.invoke(initial_state)
        
        print("\n📋 RESULTS:")
        for message in result["messages"]:
            print(message)
        print("=" * 50)
        
        return result
        
    except Exception as e:
        print(f"💥 Error running system: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test the system
if __name__ == "__main__":
    print("🧪 TESTING ORDER SYSTEM WITH REAL GEMINI\n")
    
    # Uncomment to visualize
    # visualize_graph()
    
    # Test cases
    run_order_system("I want to buy bananas")
    # run_order_system("I want to buy bananas")

🧪 TESTING ORDER SYSTEM WITH REAL GEMINI

🚀 Starting order system...
User Query: Hello
🏗️ Building graph...
✅ Graph built successfully
⚡ Executing graph...
🚦 Routing query...
✅ Routed to: greeting
🔀 Routing decision: greeting
👋 Handling greeting...
✅ Greeting handled
🏁 Ending conversation...
✅ Conversation ended

📋 RESULTS:
Routing query to: greeting
Response: Hello there! 👋 How can I help you with ordering products today?
Feel free to ask if you'd like to order anything!
